In [ ]:
from google.colab import drive
drive.mount("/content/data", force_remount=True)

Mounted at /content/data


In [ ]:
import numpy as np
import glob
from music21 import converter, instrument, note, chord, stream

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras import utils

import pickle
import os
notes = []

path = "/content/data/MyDrive/data/notes"
if os.path.exists(path):
    with open(path, "rb") as notes:
        notes = pickle.load(notes)
else:
    for file in glob.glob("midi_songs/*.mid"):
        # create a stream object
        midi = converter.parse(file)
        notes_to_parse = None

        # get all the notes and chords in the file
        parts = instrument.partitionByInstrument(midi)

        # Has instrument parts
        try:
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes
        # if parts:
        #     notes_to_parse = parts.parts[0].recurse()
        # # File has notes in flat
        # else:
        #     notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

        with open(path, "wb") as data_file:
            pickle.dump(notes, data_file)


In [ ]:

n_vocab = len(set(notes))

# Mapping from note and chord (str) to number

# all pitches name
pitch_names = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitch_names))
# Create input sequences and corresponding outputs for network
sequence_len = 100

network_input = []
network_output = []

for i in range(0, len(notes) - sequence_len, 1):
    sequence_input = notes[i:i+sequence_len]
    sequence_output = notes[i+sequence_len]
    network_input.append([note_to_int[note] for note in sequence_input])
    network_output.append(note_to_int[sequence_output])


n_patterns = len(network_input)

# reshape input to working with LSTM
network_input = np.reshape(network_input, (n_patterns, sequence_len, 1))

# Normalize inout
network_input = network_input / float(n_vocab)

# Convert input to categorical
network_output = utils.to_categorical(network_output)

network_input.shape

(5399, 100, 1)

In [ ]:
# Build Model

model = Sequential((
    LSTM(units=512,
         input_shape=(network_input.shape[1], network_input.shape[2]),
         recurrent_dropout=.3, return_sequences=True),
    LSTM(units=512,
         return_sequences=True, recurrent_dropout=.3),
    LSTM(512),
    BatchNormalization(),
    Dropout(.3),
    Dense(256),
    Activation("relu"),
    BatchNormalization(),
    Dropout(.3),
    Dense(n_vocab),
    Activation("softmax")
))

model.compile(optimizer='rmsprop', loss="categorical_crossentropy")

In [ ]:
checkpoint_path = "/content/data/MyDrive/checkpoints/music_train/model_checkpoint"

checkpoint = ModelCheckpoint(checkpoint_path, monitor="loss", save_best_only=True)

if os.path.exists(checkpoint_path):
    model = tf.keras.models.load_model(checkpoint_path)
    print("Restored model!")


model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=[checkpoint])

Restored model!
Epoch 1/200
43/43 [==============================] - 35s 547ms/step - loss: 2.1820


INFO:tensorflow:Assets written to: /content/data/MyDrive/checkpoints/music_train/model_checkpoint/assets


INFO:tensorflow:Assets written to: /content/data/MyDrive/checkpoints/music_train/model_checkpoint/assets


Epoch 2/200
43/43 [==============================] - 24s 565ms/step - loss: 2.0549


INFO:tensorflow:Assets written to: /content/data/MyDrive/checkpoints/music_train/model_checkpoint/assets


...

INFO:tensorflow:Assets written to: /content/data/MyDrive/checkpoints/music_train/model_checkpoint/assets


Epoch 165/200
43/43 [==============================] - 23s 531ms/step - loss: 0.0744
Epoch 166/200
43/43 [==============================] - 23s 534ms/step - loss: 0.0736
Epoch 167/200
43/43 [==============================] - 23s 530ms/step - loss: 0.0626
Epoch 168/200
43/43 [==============================] - 23s 528ms/step - loss: 0.0736
Epoch 169/200
43/43 [==============================] - 23s 531ms/step - loss: 0.0661
Epoch 170/200
43/43 [==============================] - 23s 524ms/step - loss: 0.0647
Epoch 171/200
43/43 [==============================] - 23s 531ms/step - loss: 0.0651
Epoch 172/200
43/43 [==============================] - 23s 529ms/step - loss: 0.1439
Epoch 173/200
43/43 [==============================] - 23s 526ms/step - loss: 0.0765
Epoch 174/200
43/43 [==============================] - 23s 528ms/step - loss: 0.0585
Epoch 175/200
43/43 [==============================] - 23s 525ms/step - loss: 0.0760
Epoch 176/200
43/43 [==============================] - 23s 529ms/

INFO:tensorflow:Assets written to: /content/data/MyDrive/checkpoints/music_train/model_checkpoint/assets


INFO:tensorflow:Assets written to: /content/data/MyDrive/checkpoints/music_train/model_checkpoint/assets


Epoch 180/200
43/43 [==============================] - 23s 530ms/step - loss: 0.0677
Epoch 181/200
43/43 [==============================] - 23s 529ms/step - loss: 0.0620
Epoch 182/200
43/43 [==============================] - 23s 524ms/step - loss: 0.0933
Epoch 183/200
43/43 [==============================] - 23s 529ms/step - loss: 0.0750
Epoch 184/200
43/43 [==============================] - 23s 529ms/step - loss: 0.0805
Epoch 185/200
43/43 [==============================] - 22s 523ms/step - loss: 0.0811
Epoch 186/200
43/43 [==============================] - 23s 533ms/step - loss: 0.0722
Epoch 187/200
43/43 [==============================] - 22s 520ms/step - loss: 0.0717
Epoch 188/200
43/43 [==============================] - 23s 530ms/step - loss: 0.0776
Epoch 189/200
43/43 [==============================] - 23s 527ms/step - loss: 0.0975
Epoch 190/200
43/43 [==============================] - 23s 526ms/step - loss: 0.0938
Epoch 191/200
43/43 [==============================] - 23s 530ms/

In [ ]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return network_input, normalized_input

In [ ]:

def generate_notes(model, network_input, pitchnames, n_vocab):
    """
    Generate notes from trained network
    :param model:
    :param network_input:
    :param pitchnames:
    :param n_vocab:
    :return:
    """
    start = np.random.randint(0, len(network_input) - 1)

    int_to_note = dict((index, note) for index, note in enumerate(pitchnames))

    pattern = network_input[start]
    predict_output = []

    for note_index in range(500):
        predict_input = np.reshape(pattern, (1, len(pattern), 1))
        predict_input = predict_input / float(n_vocab)
        prediction = model.predict(predict_input)

        index = np.argmax(prediction)
        result = int_to_note[index]
        predict_output.append(result)

        pattern.append(index)
        # np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return predict_output




In [ ]:

def create_midi(prediction_output):
    """
    Generate midi from prediction notes
    :param prediction_output:
    :return:
    """

    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ("." in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')

            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storeInstrument = instrument.Piano()
                notes.append(new_note)

            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)

        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storeInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += .5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp="midi_output.mid")



In [ ]:
pitchnames = sorted(set(item for item in notes))
# Get all pitch names
n_vocab = len(set(notes))

network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
prediction_output = generate_notes(model, network_input, pitch_names, n_vocab)
create_midi(prediction_output)

In [ ]:
!zip -r "/content/model_checkpoint.zip" "/content/model_checkpoint"

	zip warning: name not matched: /content/model_checkpoint

zip error: Nothing to do! (try: zip -r /content/model_checkpoint.zip . -i /content/model_checkpoint)


In [ ]:
from google.colab import files

files.download("/content/model_checkpoint.zip")

FileNotFoundError: ignored